<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=2fb8b3dd9b12d239d8ebaef7ec956b014a75b417712d5f61401d154c3c23f309
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-14 14:22:55
-------------------
qualified stocks: 86
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  1.09 L
Current:  1.38 C
-------------------
Today PnL: -2.13 L (-1.52%)
Current PnL: -23.36 L (-15.56%)
CY Booked + Current PnL: -11.46 L (-7.63%)
-------------------
Total profit:  1.08 L
Total loss:  -24.44 L
-------------------
Total Booked + Current PnL: 15.52 L (12.61%)
Total Booked PnL: 38.88 L (31.58%)
Curr Year Booked PnL: 11.90 L (8.62%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 90.56 L (65.56%)
Deployed:  1.23 C
Current:  1.38 C
CAGR/XIRR %: 6.84%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TTKPRESTIG,770.00,97.87,53.0,M-SC,1.27,85288.0,-15489.0,15582.0,2.14,-15.37,18.27,0.09,245.0,-0.99,0.62,12.05,OX40N,NTT,DURABLES
18,CIPLA,1662.74,-20.81,53.0,X-LC,3.73,211501.0,7001.0,16286.0,-1.27,3.42,7.70,11.39,10.0,0.43,1.54,12.68,X40N,ATH,PHARMA
83,VOLTAS,1530.00,-0.66,53.0,X-MC,1.03,208335.0,16593.0,21167.0,0.75,8.65,10.16,19.69,99.0,0.78,1.52,16.04,XY25,NTT,AC
56,RELIANCE,1533.00,-14.41,46.0,X-LC,2.85,214235.0,3869.0,24916.0,-0.12,1.84,11.63,13.68,37.0,0.16,1.56,18.67,XY25,NTT,REFINERIES
41,ITC,452.00,-40.03,38.0,X-LC,2.75,194133.0,-6005.0,26887.0,-0.56,-3.00,13.85,10.44,4.0,-0.22,1.42,2.40,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
66,SONACOMS,1006.0,-36.16,64.0,M-SC,12.85,79389.0,-21770.0,96664.0,4.11,-21.52,121.76,74.03,202.0,-0.23,0.58,11.95,AR,BTT,AUTO
76,TTKPRESTIG,770.0,97.87,53.0,M-SC,1.27,85288.0,-15489.0,15582.0,2.14,-15.37,18.27,0.09,245.0,-0.99,0.62,12.05,OX40N,NTT,DURABLES
4,ANGELONE,3033.0,10.20,62.0,X-SC,10.33,195242.0,4236.0,53457.0,1.72,2.22,27.38,30.21,157.0,0.08,1.42,23.56,X40N,NTT,FINANCE
85,WIPRO,420.0,-11.98,55.0,M-LC,5.93,154293.0,3348.0,106107.0,1.52,2.22,68.77,72.51,53.0,0.03,1.13,8.37,XR,NTT,IT
17,CERA,9475.0,-23.15,44.0,H-SC,1.96,140254.0,-35649.0,77673.0,1.43,-20.27,55.38,23.89,149.0,-0.46,1.02,21.13,OX40N,NTT,CERAMICS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.0,-54.49,12.0,X-LC,1.07,144837.0,-131686.0,241762.0,-39.61,-47.62,166.92,39.81,54.0,-0.54,1.06,0.00,XY24,NTT,AUTO
81,VALIANTORG,838.0,-277.79,23.0,H-SC,3.11,98838.0,-64767.0,186082.0,-4.84,-39.59,188.27,74.15,139.0,-0.35,0.72,26.69,XR,NTT,CHEMICALS
23,DEN,75.0,89.12,36.0,M-SC,3.88,52800.0,-31664.0,67199.0,-4.13,-37.49,127.27,42.07,237.0,-0.47,0.39,12.13,AR,NTT,ENTERTAINMENT
10,BANDHANBNK,400.0,-13.31,47.0,H-SC,2.95,217160.0,-61399.0,318834.0,-3.81,-22.04,146.82,92.42,151.0,-0.19,1.58,25.50,XY24,NTT,BANKS
68,SURYODAY,240.0,57.89,53.0,H-SC,10.44,144993.0,-34078.0,99813.0,-3.55,-19.03,68.84,36.71,135.0,-0.34,1.06,43.86,XR,NTT,BANKS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,INDIAMART,4810.62,-55.76,36.0,H-SC,8.29,123352.0,16.0,131617.0,-2.96,0.01,106.70,106.72,119.0,0.00,0.90,23.39,AR,ATH,MISC
54,RECLTD,446.00,45.36,43.0,M-LC,6.20,203885.0,715.0,41409.0,-0.11,0.35,20.31,20.74,55.0,0.02,1.49,5.91,XY25,NTT,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,104.20,56.0,H-SC,11.54,128925.0,-13554.0,33069.0,-2.03,-9.51,25.65,13.70,163.0,-0.41,0.94,52.17,OX40N,NTT,BANKS
76,TTKPRESTIG,770.0,97.87,53.0,M-SC,1.27,85288.0,-15489.0,15582.0,2.14,-15.37,18.27,0.09,245.0,-0.99,0.62,12.05,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.97,48.0,H-SC,2.06,220608.0,-49059.0,85397.0,-0.24,-18.19,38.71,13.47,138.0,-0.57,1.61,12.17,XY24,NTT,PAINTS
17,CERA,9475.0,-23.15,44.0,H-SC,1.96,140254.0,-35649.0,77673.0,1.43,-20.27,55.38,23.89,149.0,-0.46,1.02,21.13,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-25.22,42.0,H-MC,7.03,101251.0,-30584.0,72810.0,-0.43,-23.20,71.91,32.03,98.0,-0.42,0.74,13.41,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RECLTD,446.00,45.36,43.0,M-LC,6.20,203885.0,715.0,41409.0,-0.11,0.35,20.31,20.74,55.0,0.02,1.49,5.91,XY25,NTT,FINANCE
31,HINDZINC,730.22,34.91,66.0,M-LC,8.55,219610.0,14534.0,98034.0,-0.14,7.09,44.64,54.89,52.0,0.15,1.60,32.89,X5K,ATH,METALS
85,WIPRO,420.00,-11.98,55.0,M-LC,5.93,154293.0,3348.0,106107.0,1.52,2.22,68.77,72.51,53.0,0.03,1.13,8.37,XR,NTT,IT
78,UNIONBANK,163.00,-13.18,51.0,M-LC,9.50,155928.0,15088.0,29408.0,-1.97,10.71,18.86,31.59,66.0,0.51,1.14,40.04,XY24,NTT,BANKS
11,BANKINDIA,190.00,-26.71,57.0,H-MC,12.64,190420.0,10612.0,101799.0,-2.40,5.90,53.46,62.52,88.0,0.10,1.39,42.00,XR,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HINDZINC,730.22,34.91,66.0,M-LC,8.55,219610.0,14534.0,98034.0,-0.14,7.09,44.64,54.89,52.0,0.15,1.60,32.89,X5K,ATH,METALS
11,BANKINDIA,190.00,-26.71,57.0,H-MC,12.64,190420.0,10612.0,101799.0,-2.40,5.90,53.46,62.52,88.0,0.10,1.39,42.00,XR,NTT,BANKS
85,WIPRO,420.00,-11.98,55.0,M-LC,5.93,154293.0,3348.0,106107.0,1.52,2.22,68.77,72.51,53.0,0.03,1.13,8.37,XR,NTT,IT
36,INDIAMART,4810.62,-55.76,36.0,H-SC,8.29,123352.0,16.0,131617.0,-2.96,0.01,106.70,106.72,119.0,0.00,0.90,23.39,AR,ATH,MISC
84,WHIRLPOOL,2270.00,-49.00,29.0,M-SC,3.42,86205.0,-5294.0,84041.0,-1.27,-5.79,97.49,86.07,223.0,-0.06,0.63,25.19,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-54.49,12.0,X-LC,1.07,144837.0,-131686.0,241762.0,-39.61,-47.62,166.92,39.81,54.0,-0.54,1.06,0.00,XY24,NTT,AUTO
28,HAVELLS,2069.16,-9.01,28.0,X-MC,1.38,225607.0,-22124.0,97191.0,-1.83,-8.93,43.08,30.30,92.0,-0.23,1.65,3.17,X40,ATH,ELECTRICAL
51,QUESS,986.00,-48.21,29.0,X-SC,37.85,51366.0,-13640.0,164566.0,-1.66,-20.98,320.38,232.18,198.0,-0.08,0.37,0.00,XY24,NTT,MISC
9,BAJAJHFL,181.50,-16.60,29.0,X-MC,5.52,175956.0,-24550.0,121709.0,-1.32,-12.24,69.17,48.45,90.0,-0.20,1.28,1.47,X40N,ATH,FINANCE
5,ASIANPAINT,3465.66,-19.07,31.0,X-LC,5.31,204186.0,-47582.0,100786.0,-0.95,-18.90,49.36,21.13,27.0,-0.47,1.49,9.83,X40,ATH,PAINTS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,4671.50,-3.90,41.0,H-LC,0.92,154605.0,-31490.0,78972.0,0.47,-16.92,51.08,25.51,15.0,-0.40,1.13,13.93,AR,ATH,ELECTRICAL
83,VOLTAS,1530.00,-0.66,53.0,X-MC,1.03,208335.0,16593.0,21167.0,0.75,8.65,10.16,19.69,99.0,0.78,1.52,16.04,XY25,NTT,AC
72,TATAMOTORS,1065.00,-54.49,12.0,X-LC,1.07,144837.0,-131686.0,241762.0,-39.61,-47.62,166.92,39.81,54.0,-0.54,1.06,0.00,XY24,NTT,AUTO
28,HAVELLS,2069.16,-9.01,28.0,X-MC,1.38,225607.0,-22124.0,97191.0,-1.83,-8.93,43.08,30.30,92.0,-0.23,1.65,3.17,X40,ATH,ELECTRICAL
33,ICICIGI,2260.25,-20.58,41.0,X-MC,1.46,136545.0,412.0,30709.0,0.64,0.30,22.49,22.86,91.0,0.01,1.00,12.95,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-48.21,29.0,X-SC,37.85,51366.0,-13640.0,164566.0,-1.66,-20.98,320.38,232.18,198.0,-0.08,0.37,0.00,XY24,NTT,MISC
72,TATAMOTORS,1065.00,-54.49,12.0,X-LC,1.07,144837.0,-131686.0,241762.0,-39.61,-47.62,166.92,39.81,54.0,-0.54,1.06,0.00,XY24,NTT,AUTO
9,BAJAJHFL,181.50,-16.60,29.0,X-MC,5.52,175956.0,-24550.0,121709.0,-1.32,-12.24,69.17,48.45,90.0,-0.20,1.28,1.47,X40N,ATH,FINANCE
82,VBL,671.64,-21.70,38.0,X-LC,10.50,281964.0,-33878.0,146537.0,-0.11,-10.73,51.97,35.67,5.0,-0.23,2.06,1.64,X40N,ATH,FMCG
21,COLPAL,3767.14,-5.64,35.0,X-MC,7.68,220100.0,-43265.0,156623.0,-0.91,-16.43,71.16,43.04,84.0,-0.28,1.61,2.18,XY25,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-48.21,29.0,X-SC,37.85,51366.0,-13640.0,164566.0,-1.66,-20.98,320.38,232.18,198.0,-0.08,0.37,0.00,XY24,NTT,MISC
55,RELAXO,1176.00,-41.04,35.0,X-SC,3.32,80332.0,-64828.0,140758.0,-1.08,-44.66,175.22,52.31,136.0,-0.46,0.59,7.53,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-37.58,34.0,X-SC,4.36,93292.0,-35378.0,80679.0,-2.18,-27.50,86.48,35.20,219.0,-0.44,0.68,7.08,X40,NTT,FOOTWEAR
16,CAMS,5211.76,-6.94,43.0,X-SC,1.05,105871.0,3867.0,40062.0,-2.25,3.79,37.84,43.06,122.0,0.10,0.77,21.56,X40N,ATH,MISC
32,HONAUT,58357.33,-23.25,44.0,X-SC,2.23,108120.0,-19818.0,66948.0,-0.32,-15.49,61.92,36.84,143.0,-0.30,0.79,10.69,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4406.34,-30.29,43.0,X-LC,12.30,278372.0,-67584.0,135818.0,-1.52,-19.54,48.79,19.73,1.0,-0.50,2.03,2.53,X40,ATH,IT
39,INFY,2275.00,-18.16,52.0,X-LC,7.50,318499.0,5611.0,166065.0,0.14,1.79,52.14,54.87,3.0,0.03,2.32,8.56,X40,BTT,IT
41,ITC,452.00,-40.03,38.0,X-LC,2.75,194133.0,-6005.0,26887.0,-0.56,-3.00,13.85,10.44,4.0,-0.22,1.42,2.40,X40,NTT,FMCG
82,VBL,671.64,-21.70,38.0,X-LC,10.50,281964.0,-33878.0,146537.0,-0.11,-10.73,51.97,35.67,5.0,-0.23,2.06,1.64,X40N,ATH,FMCG
1,ABB,7934.00,-41.23,41.0,H-LC,7.29,245616.0,-16003.0,135212.0,-0.59,-6.12,55.05,45.57,7.0,-0.12,1.79,4.51,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
20,COFFEEDAY,80.00,-45.67,48.0,L-SC,28.71,80832.0,-32717.0,72765.0,-1.15,-28.81,90.02,35.27,268.0,-0.45,0.59,97.84,XR,NTT,HOTELS
48,MASFIN,398.61,-21.86,37.0,H-SC,9.90,88845.0,-9135.0,30740.0,-1.89,-9.32,34.60,22.05,152.0,-0.30,0.65,29.14,XR,ATH,FINANCE
77,UJJIVANSFB,60.00,104.20,56.0,H-SC,11.54,128925.0,-13554.0,33069.0,-2.03,-9.51,25.65,13.70,163.0,-0.41,0.94,52.17,OX40N,NTT,BANKS
68,SURYODAY,240.00,57.89,53.0,H-SC,10.44,144993.0,-34078.0,99813.0,-3.55,-19.03,68.84,36.71,135.0,-0.34,1.06,43.86,XR,NTT,BANKS
60,SAMMAANCAP,326.00,-185.23,67.0,M-SC,17.48,147690.0,-1530.0,145711.0,1.04,-1.03,98.66,96.62,208.0,-0.01,1.08,59.83,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-185.23,67.0,M-SC,17.48,147690.0,-1530.0,145711.0,1.04,-1.03,98.66,96.62,208.0,-0.01,1.08,59.83,XY25,NTT,FINANCE
68,SURYODAY,240.00,57.89,53.0,H-SC,10.44,144993.0,-34078.0,99813.0,-3.55,-19.03,68.84,36.71,135.0,-0.34,1.06,43.86,XR,NTT,BANKS
31,HINDZINC,730.22,34.91,66.0,M-LC,8.55,219610.0,14534.0,98034.0,-0.14,7.09,44.64,54.89,52.0,0.15,1.60,32.89,X5K,ATH,METALS
50,PGHH,17907.65,-26.31,61.0,X-MC,2.91,213150.0,12330.0,55462.0,0.22,6.14,26.02,33.76,80.0,0.22,1.56,11.66,X40,ATH,FMCG
63,SHALBY,327.00,1161.56,52.0,M-SC,22.89,171621.0,-9522.0,54987.0,-2.83,-5.26,32.04,25.10,235.0,-0.17,1.25,37.54,XY24,NTT,HEALTHCARE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.48
1,25,44.00
2,50,75.81


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.42
LC    33.70
MC    21.87
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.66
X40      14.86
XY25     12.12
XR       10.97
X40N     10.09
AR        8.24
OX40N     7.70
X200      1.80
X5K       1.60
SR        1.12
MH        0.83
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.62
X-LC    22.59
X-MC    16.14
M-SC    12.74
M-LC     5.36
H-LC     4.72
X-SC     4.62
H-MC     3.80
M-MC     1.62
L-SC     1.44
L-LC     1.03
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.07,-6.54,41.65
IT,13.08,-17.31,80.85
FINANCE,12.17,-13.32,64.69
MISC,6.90,-21.20,88.36
BANKS,6.42,-13.38,73.03
PAINTS,5.62,-19.94,37.81
ELECTRICAL,5.36,-12.19,51.55
INSURANCE,3.85,-5.08,44.36
AUTO,2.84,-47.75,118.39


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3371235.0,22
XR,1407246.0,14
AR,1200826.0,9
X40,805057.0,10
X40N,643892.0,8
OX40N,589648.0,10
XY25,522686.0,8
SR,268444.0,2
X5K,98034.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3487460.0,25
M-SC,1295253.0,15
X-LC,1145115.0,13
X-MC,1116651.0,11
X-SC,546470.0,6
H-MC,359690.0,3
H-LC,292706.0,3
M-LC,274958.0,4
L-SC,263639.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1215173.0      6
           XR         832949.0      7
           AR         822779.0      5
M-SC       XY24       703535.0      6
X-MC       XY24       590692.0      4
X-LC       X40        497918.0      5
           XY24       311817.0      2
X-SC       X40N       301225.0      4
H-SC       OX40N      277600.0      4
           SR         268444.0      2
X-MC       X40        226460.0      4
X-LC       X40N       220958.0      3
H-LC       AR         214184.0      2
H-MC       XY24       185081.0      1
X-MC       XY25       177790.0      2
L-SC       XR         173527.0      2
M-MC       XY24       170963.0      1
X-SC       XY24       164566.0      1
M-SC       AR         163863.0      2
           OX40N      149126.0      4
           XY25       145711.0      1
           XR         133018.0      2
X-MC       X40N       121709.0      1
X-LC       XY25       114422.0      3
M-LC       XR         106107.0      1
H-MC       XR         101799.0      1
M-LC       X5K         98034.0      1
L-SC       OX40N       90112.0      1
X-SC       X40         80679.0      1
H-LC       X200        78522.0      1
H-MC       OX40N       72810.0      1
H-SC       MH          70515.0      1
L-MC       XR          59846.0      1
L-LC       XY25        43354.0      1
M-LC       XY25        41409.0      1
           XY24        29408.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 31.0 seconds
